In [4]:
import pandas as pd

# Load the datasets
circuits = pd.read_csv('historic_data/circuits.csv')
constructor_results = pd.read_csv('historic_data/constructor_results.csv')
constructor_standings = pd.read_csv('historic_data/constructor_standings.csv')
constructors = pd.read_csv('historic_data/constructors.csv')
driver_standings = pd.read_csv('historic_data/driver_standings.csv')
drivers = pd.read_csv('historic_data/drivers.csv')
pit_stops = pd.read_csv('historic_data/pit_stops.csv')
qualifying = pd.read_csv('historic_data/qualifying.csv')
races = pd.read_csv('historic_data/races.csv')
results = pd.read_csv('historic_data/results.csv')
seasons = pd.read_csv('historic_data/seasons.csv')
sprint_results = pd.read_csv('historic_data/sprint_results.csv')
status = pd.read_csv('historic_data/status.csv')

# Display the first few rows of each dataset to understand their structure
datasets_preview = {
    "circuits": circuits.head(),
    "constructor_results": constructor_results.head(),
    "constructor_standings": constructor_standings.head(),
    "constructors": constructors.head(),
    "driver_standings": driver_standings.head(),
    "drivers": drivers.head(),
    "pit_stops": pit_stops.head(),
    "qualifying": qualifying.head(),
    "races": races.head(),
    "results": results.head(),
    "seasons": seasons.head(),
    "sprint_results": sprint_results.head(),
    "status": status.head()
}

# Display datasets preview
for name, df in datasets_preview.items():
    print(f"Dataset: {name}")
    print(df)
    print("\n")


Dataset: circuits
   circuitId   circuitRef                            name      location  \
0          1  albert_park  Albert Park Grand Prix Circuit     Melbourne   
1          2       sepang    Sepang International Circuit  Kuala Lumpur   
2          3      bahrain   Bahrain International Circuit        Sakhir   
3          4    catalunya  Circuit de Barcelona-Catalunya      Montmeló   
4          5     istanbul                   Istanbul Park      Istanbul   

     country       lat        lng  alt  \
0  Australia -37.84970  144.96800   10   
1   Malaysia   2.76083  101.73800   18   
2    Bahrain  26.03250   50.51060    7   
3      Spain  41.57000    2.26111  109   
4     Turkey  40.95170   29.40500  130   

                                                 url  
0  http://en.wikipedia.org/wiki/Melbourne_Grand_P...  
1  http://en.wikipedia.org/wiki/Sepang_Internatio...  
2  http://en.wikipedia.org/wiki/Bahrain_Internati...  
3  http://en.wikipedia.org/wiki/Circuit_de_Barcel...  
4  

In [6]:
# Filter races to include only 2021, 2022, and 2023 seasons
races_filtered = races[races['year'].isin([2021, 2022, 2023])]

# Filter relevant data from each dataset based on the filtered races
relevant_race_ids = races_filtered['raceId']

constructor_results_filtered = constructor_results[constructor_results['raceId'].isin(relevant_race_ids)]
constructor_standings_filtered = constructor_standings[constructor_standings['raceId'].isin(relevant_race_ids)]
driver_standings_filtered = driver_standings[driver_standings['raceId'].isin(relevant_race_ids)]
pit_stops_filtered = pit_stops[pit_stops['raceId'].isin(relevant_race_ids)]
qualifying_filtered = qualifying[qualifying['raceId'].isin(relevant_race_ids)]
results_filtered = results[results['raceId'].isin(relevant_race_ids)]
sprint_results_filtered = sprint_results[sprint_results['raceId'].isin(relevant_race_ids)]

# Select relevant columns for merging
circuits_relevant = circuits[['circuitId', 'name', 'location', 'country']]
constructors_relevant = constructors[['constructorId', 'name']]
drivers_relevant = drivers[['driverId', 'surname', 'nationality']]
status_relevant = status[['statusId', 'status']]
races_relevant = races_filtered[['raceId', 'year', 'round', 'circuitId', 'date', 'time']]

# Merge datasets step-by-step to create a comprehensive dataset
# Start with races and circuits
merged_data = pd.merge(races_relevant, circuits_relevant, on='circuitId')

# Add results
merged_data = pd.merge(merged_data, results_filtered, on='raceId')

# Add drivers information
merged_data = pd.merge(merged_data, drivers_relevant, on='driverId')

# Add constructors information
merged_data = pd.merge(merged_data, constructors_relevant, on='constructorId')

# Add driver standings
merged_data = pd.merge(merged_data, driver_standings_filtered[['raceId', 'driverId', 'points', 'position']], on=['raceId', 'driverId'], suffixes=('', '_driver_standings'))

# Add constructor standings
merged_data = pd.merge(merged_data, constructor_standings_filtered[['raceId', 'constructorId', 'points', 'position']], on=['raceId', 'constructorId'], suffixes=('', '_constructor_standings'))

# Add qualifying results
merged_data = pd.merge(merged_data, qualifying_filtered[['raceId', 'driverId', 'position']], on=['raceId', 'driverId'], suffixes=('', '_qualifying'))

# Add sprint results
merged_data = pd.merge(merged_data, sprint_results_filtered[['raceId', 'driverId', 'positionOrder', 'points']], on=['raceId', 'driverId'], suffixes=('', '_sprint'))

# Add status information
merged_data = pd.merge(merged_data, status_relevant, on='statusId')

print(merged_data)

     raceId  year  round  circuitId        date    time_x  \
0      1061  2021     10          9  2021-07-18  14:00:00   
1      1061  2021     10          9  2021-07-18  14:00:00   
2      1061  2021     10          9  2021-07-18  14:00:00   
3      1061  2021     10          9  2021-07-18  14:00:00   
4      1061  2021     10          9  2021-07-18  14:00:00   
..      ...   ...    ...        ...         ...       ...   
175    1110  2023     12         13  2023-07-30  13:00:00   
176    1110  2023     12         13  2023-07-30  13:00:00   
177    1110  2023     12         13  2023-07-30  13:00:00   
178    1110  2023     12         13  2023-07-30  13:00:00   
179    1110  2023     12         13  2023-07-30  13:00:00   

                           name_x     location  country  resultId  ...  \
0             Silverstone Circuit  Silverstone       UK     25146  ...   
1             Silverstone Circuit  Silverstone       UK     25147  ...   
2             Silverstone Circuit  Silverston

In [18]:
import pandas as pd

# Drop unnecessary columns and rename for clarity
columns_to_keep_corrected = [
    'raceId', 'year', 'round', 'circuitId', 'date', 'time_x', 'name_x', 'location', 'country',
    'driverId', 'surname', 'nationality', 'constructorId', 'name_y', 'grid', 'positionOrder',
    'points', 'laps', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'points_driver_standings',
    'position_driver_standings', 'points_constructor_standings', 'position_constructor_standings',
    'position_qualifying', 'positionOrder_sprint', 'points_sprint', 'status'
]

# Renaming columns for clarity
merged_data_renamed = merged_data.rename(columns={
    'time_x': 'race_time',
    'name_x': 'circuit_name',
    'name_y': 'constructor_name'
})

# Select the renamed columns
final_dataset_corrected = merged_data_renamed[[
    'raceId', 'year', 'round', 'circuitId', 'date', 'race_time', 'circuit_name', 'location', 'country',
    'driverId', 'surname', 'nationality', 'constructorId', 'constructor_name', 'grid', 'positionOrder',
    'points', 'laps', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'points_driver_standings',
    'position_driver_standings', 'points_constructor_standings', 'position_constructor_standings',
    'position_qualifying', 'positionOrder_sprint', 'points_sprint', 'status'
]]

# Convert date and race_time columns to numeric formats
final_dataset_corrected['date'] = pd.to_datetime(final_dataset_corrected['date']).astype(int) // 10**9  # Convert to Unix timestamp

def convert_to_seconds(time_str):
    try:
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s
    except:
        return 0

final_dataset_corrected['race_time'] = final_dataset_corrected['race_time'].apply(convert_to_seconds)

# Convert milliseconds, fastestLap, rank, and fastestLapTime to numeric if possible
final_dataset_corrected['milliseconds'] = pd.to_numeric(final_dataset_corrected['milliseconds'], errors='coerce')
final_dataset_corrected['fastestLap'] = pd.to_numeric(final_dataset_corrected['fastestLap'], errors='coerce')
final_dataset_corrected['rank'] = pd.to_numeric(final_dataset_corrected['rank'], errors='coerce')
final_dataset_corrected['fastestLapTime'] = pd.to_numeric(final_dataset_corrected['fastestLapTime'].str.replace(':', ''), errors='coerce')

# Set missing fastestLapTime and milliseconds values to 0
final_dataset_corrected['fastestLapTime'].fillna(0, inplace=True)
final_dataset_corrected['milliseconds'].fillna(0, inplace=True)

# Verify there are no more missing values
# missing_values_after_imputation = final_dataset_corrected.isnull().sum()
# print("Missing Values After Imputation:\n", missing_values_after_imputation)

print(final_dataset_corrected)
# Save the cleaned dataset
final_dataset_corrected.to_csv('cleaned_data/final_dataset.csv', index=False)


     raceId  year  round  circuitId        date  race_time  \
0      1061  2021     10          9  1626566400      50400   
1      1061  2021     10          9  1626566400      50400   
2      1061  2021     10          9  1626566400      50400   
3      1061  2021     10          9  1626566400      50400   
4      1061  2021     10          9  1626566400      50400   
..      ...   ...    ...        ...         ...        ...   
175    1110  2023     12         13  1690675200      46800   
176    1110  2023     12         13  1690675200      46800   
177    1110  2023     12         13  1690675200      46800   
178    1110  2023     12         13  1690675200      46800   
179    1110  2023     12         13  1690675200      46800   

                     circuit_name     location  country  driverId  ... rank  \
0             Silverstone Circuit  Silverstone       UK         1  ...    2   
1             Silverstone Circuit  Silverstone       UK       844  ...    5   
2             Silv

/var/folders/nq/x683qlmx39ng8v_l8ymc6cw00000gn/T/ipykernel_28447/70182085.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataset_corrected['date'] = pd.to_datetime(final_dataset_corrected['date']).astype(int) // 10**9  # Convert to Unix timestamp
/var/folders/nq/x683qlmx39ng8v_l8ymc6cw00000gn/T/ipykernel_28447/70182085.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataset_corrected['race_time'] = final_dataset_corrected['race_time'].apply(convert_to_seconds)
/var/folders/nq/x683qlmx